In [1]:
import pandas as pd
from nltk import word_tokenize

import setup_modules
from lib.fomc import get_lname

In [2]:
df = pd.read_csv('../working-csvs/raw_transcripts_sectioned.csv', index_col=0)

In [3]:
df['lname'] = df['speaker'].map(get_lname)
df['len'] = df['content'].map(lambda x : len(word_tokenize(x)))

In [4]:
fomc = pd.read_csv('../working-csvs/fomc.csv', index_col=0)
fm = df.merge(fomc, how='inner', left_on=['date', 'lname'], right_on=['date', 'member'])

In [5]:
fm

,speaker,content,index,date,type,section,lname,len,member,voter,region,Econ_PhD,female,chair,exp
0,CHAIRMAN GREENSPAN.,"I want to note that revisions, if any, on the ...",72,1994-02-04,meeting,1,greenspan,35,greenspan,1,governor,1,0.0,Greenspan,6.490411
1,CHAIRMAN GREENSPAN.,This is known as the statistical sausage factory!,78,1994-02-04,meeting,1,greenspan,9,greenspan,1,governor,1,0.0,Greenspan,6.490411
2,CHAIRMAN GREENSPAN.,Interest and dividends versus wages and salaries?,81,1994-02-04,meeting,1,greenspan,8,greenspan,1,governor,1,0.0,Greenspan,6.490411
3,CHAIRMAN GREENSPAN.,It's not mobile homes; this is housing that is...,88,1994-02-04,meeting,1,greenspan,16,greenspan,1,governor,1,0.0,Greenspan,6.490411
4,CHAIRMAN GREENSPAN.,"Yes, a lot of that type of construction goes o...",90,1994-02-04,meeting,1,greenspan,21,greenspan,1,governor,1,0.0,Greenspan,6.490411
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32700,MS. GEORGE.,"That said, raising the funds rate at this meet...",244,2018-12-19,meeting,2,george,369,george,0,Kansas City,0,1.0,Powell,7.221918
32701,MS. BOWMAN.,"Thank you, Mr. Chairman. As many have already...",162,2018-12-19,meeting,1,bowman,970,bowman,1,governor,0,1.0,Powell,0.063014
32702,MS. BOWMAN.,"Thank you, Mr. Chairman. I also support alter...",240,2018-12-19,meeting,2,bowman,467,bowman,1,governor,0,1.0,Powell,0.063014
32703,MR. QUARLES.,"Thank you, Mr. Chairman. Notwithstanding the ...",164,2018-12-19,meeting,1,quarles,756,quarles,1,governor,0,0.0,Powell,1.254795


In [6]:
fm['longest'] = fm.groupby(['date', 'section', 'speaker'])['len'].transform('max')

In [7]:
fm['longest_utt'] = (fm['longest'] == fm['len']).astype('int')

In [8]:
fm.sort_values(['date', 'index'], inplace=True)

In [9]:
fm['order_temp'] = fm.groupby(['date', 'section'])['longest_utt'].cumsum()

In [10]:
fm['order'] = fm['order_temp'] * fm['longest_utt']

In [11]:
fm

,speaker,content,index,date,type,section,lname,len,member,voter,region,Econ_PhD,female,chair,exp,longest,longest_utt,order_temp,order
0,CHAIRMAN GREENSPAN.,"I want to note that revisions, if any, on the ...",72,1994-02-04,meeting,1,greenspan,35,greenspan,1,governor,1,0.0,Greenspan,6.490411,164,0,0,0
80,MR. LINDSEY.,I have two questions. The first has to do with...,73,1994-02-04,meeting,1,lindsey,31,lindsey,1,governor,1,0.0,Greenspan,2.194521,619,0,0,0
81,MR. LINDSEY.,Could I get a copy of that breakdown?,75,1994-02-04,meeting,1,lindsey,9,lindsey,1,governor,1,0.0,Greenspan,2.194521,619,0,0,0
82,MR. LINDSEY.,Horrors!,77,1994-02-04,meeting,1,lindsey,2,lindsey,1,governor,1,0.0,Greenspan,2.194521,619,0,0,0
1,CHAIRMAN GREENSPAN.,This is known as the statistical sausage factory!,78,1994-02-04,meeting,1,greenspan,9,greenspan,1,governor,1,0.0,Greenspan,6.490411,164,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32631,MR. KASHKARI.,"Thank you, Mr. Chairman. If I might, I wanted...",248,2018-12-19,meeting,2,kashkari,931,kashkari,0,Minneapolis,0,0.0,Powell,2.967123,931,1,14,14
32618,CHAIRMAN POWELL.,Thank you. Vice Chair Williams.,249,2018-12-19,meeting,2,powell,7,powell,1,governor,0,0.0,Powell,6.572603,511,0,14,0
32661,VICE CHAIRMAN WILLIAMS.,"Thank you, Mr. Chairman. I support alternativ...",250,2018-12-19,meeting,2,williams,1435,williams,1,New York,1,0.0,Powell,7.805479,1435,1,15,15
32662,VICE CHAIRMAN WILLIAMS.,"Thank you, Mr. Chairman. I support alternativ...",250,2018-12-19,meeting,2,williams,1435,williams,1,New York,1,0.0,Powell,7.805479,1435,1,16,16


In [12]:
fm[(fm['date'] == '1994-02-04') & (fm['section'] == 1)].groupby('speaker')['order'].max().sort_values()
fm[(fm['date'] == '1994-02-04') & (fm['section'] == 2) & (fm['longest_utt'] == 1)]

,speaker,content,index,date,type,section,lname,len,member,voter,region,Econ_PhD,female,chair,exp,longest,longest_utt,order_temp,order
46,CHAIRMAN GREENSPAN.,"Questions for Don? If not, let me get started....",380,1994-02-04,meeting,2,greenspan,1318,greenspan,1,governor,1,0.0,Greenspan,6.490411,1318,1,1,1
174,VICE CHAIRMAN MCDONOUGH.,"Thank you, Mr. Chairman. I believe very strong...",381,1994-02-04,meeting,2,mcdonough,520,mcdonough,1,New York,0,0.0,Greenspan,0.547945,520,1,2,2
172,MR. MELZER.,"Well, as I stated yesterday, Alan, the stance ...",383,1994-02-04,meeting,2,melzer,303,melzer,0,St. Louis,0,0.0,Greenspan,8.684932,303,1,3,3
137,MR. JORDAN.,"For some time, just viewing things from my Dis...",385,1994-02-04,meeting,2,jordan,345,jordan,1,Cleveland,1,0.0,Greenspan,1.906849,345,1,4,4
143,MR. BOEHNE.,"As far as the economy goes, this is as good as...",391,1994-02-04,meeting,2,boehne,404,boehne,0,Philadelphia,1,0.0,Greenspan,13.016438,404,1,5,5
168,MR. BROADDUS.,"Mr. Chairman, I certainly think that the time ...",393,1994-02-04,meeting,2,broaddus,319,broaddus,1,Richmond,1,0.0,Greenspan,1.093151,319,1,6,6
160,MR. FORRESTAL.,"Mr. Chairman, this decision is not altogether ...",395,1994-02-04,meeting,2,forrestal,509,forrestal,1,Atlanta,0,0.0,Greenspan,10.169863,509,1,7,7
102,MR. LINDSEY.,"Well, if anything, I think the risks are that ...",403,1994-02-04,meeting,2,lindsey,417,lindsey,1,governor,1,0.0,Greenspan,2.194521,417,1,8,8
164,MR. HOENIG.,"Mr. Chairman, I think yesterday's discussion a...",411,1994-02-04,meeting,2,hoenig,95,hoenig,0,Kansas City,1,0.0,Greenspan,2.347945,95,1,9,9
170,MR. STERN.,I agree with all the comments that have been m...,413,1994-02-04,meeting,2,stern,167,stern,0,Minneapolis,1,0.0,Greenspan,8.882192,167,1,10,10


In [13]:
orderdf = fm.groupby(['date', 'section', 'lname'])['order'].max().reset_index()

In [20]:
orderdf.sort_values(['date', 'section', 'order'])[orderdf['date'] == '2018-12-19']

/tmp/ipykernel_29473/2917729530.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  orderdf.sort_values(['date', 'section', 'order'])[orderdf['date'] == '2018-12-19']


,date,section,lname,order
6678,2018-12-19,1,bostic,1
6682,2018-12-19,1,clarida,2
6691,2018-12-19,1,rosengren,3
6685,2018-12-19,1,harker,4
6677,2018-12-19,1,barkin,5
6681,2018-12-19,1,bullard,6
6688,2018-12-19,1,mester,7
6683,2018-12-19,1,evans,8
6680,2018-12-19,1,brainard,9
6686,2018-12-19,1,kaplan,10


In [15]:
orderdf.to_csv('../working-csvs/order.csv')